# 🧹 03 - Enrichissement des données OpenFoodFacts
Ce notebook a pour objectif d'enrichir les données déjà transformées en CSV et nettoyées.


In [1]:
# 0. Stoppe toute session existante
try:
    spark.stop()
except:
    pass

# Recréation SparkSession en local, FS local et driver binding fixe
from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
        .appName("03_Enrichment")
        .master("local[*]")
        .config("spark.hadoop.fs.defaultFS", "file:///")
        .config("spark.driver.host", "127.0.0.1")
        .config("spark.driver.bindAddress", "0.0.0.0")
        .getOrCreate()
)


In [2]:
# Chargement des données nettoyées depuis le CSV généré à l'étape précédente
input_path = "../data/step2_cleaned_csv"

df_cleaned = (
    spark.read
        .option("header", "true")
        .option("inferSchema", "true")
        .option("sep", ";")
        .csv(input_path)
)

print(f"✅ Données chargées : {df_cleaned.count():,} lignes, {len(df_cleaned.columns)} colonnes")


✅ Données chargées : 180,041 lignes, 1 colonnes


In [3]:
# 3. Aperçu rapide de la structure du DataFrame
df_cleaned.printSchema()

# 4. Liste complète des colonnes disponibles
colonnes = df_cleaned.columns
print("🧾 Colonnes disponibles :", colonnes)

# 5. Affichage d’un échantillon pour inspection visuelle
colonnes_extraites = [
    "brands", "brand_owner", "countries", "categories",
    "nutriscore_score", "nutriscore_grade", "nova_group",
    "energy_100g", "fat_100g", "saturated-fat_100g", "sugars_100g", "salt_100g"
]

df_cleaned.select(*colonnes_extraites).show(5, truncate=False)

root
 |-- product_name,abbreviated_product_name,generic_name,quantity,packaging,brands,categories,origins,manufacturing_places,labels,purchase_places,stores,countries,ingredients_text,ingredients_tags,ingredients_analysis_tags,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,serving_quantity,no_nutrition_data,additives_n,additives,additives_tags,additives_en,nutriscore_score,nutriscore_grade,nova_group,food_groups,brand_owner,environmental_score_score,environmental_score_grade,nutrient_levels_tags,product_quantity,popularity_tags,completeness,main_category,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,butyric-acid_100g,caproic-acid_100g,caprylic-acid_100g,capric-acid_100g,lauric-acid_100g,myristic-acid_100g,palmitic-acid_100g,stearic-acid_100g,arachidic-acid_100g,behenic-acid_100g,lignoceric-acid_100g,cerotic-acid_100g,montanic-acid_100g,melissic-acid_100g,unsaturated-fat_100g,monounsaturated-fat_100g,omega-9-fat_100g,polyu

AnalysisException: Column 'brands' does not exist. Did you mean one of the following? [product_name,abbreviated_product_name,generic_name,quantity,packaging,brands,categories,origins,manufacturing_places,labels,purchase_places,stores,countries,ingredients_text,ingredients_tags,ingredients_analysis_tags,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,serving_quantity,no_nutrition_data,additives_n,additives,additives_tags,additives_en,nutriscore_score,nutriscore_grade,nova_group,food_groups,brand_owner,environmental_score_score,environmental_score_grade,nutrient_levels_tags,product_quantity,popularity_tags,completeness,main_category,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,butyric-acid_100g,caproic-acid_100g,caprylic-acid_100g,capric-acid_100g,lauric-acid_100g,myristic-acid_100g,palmitic-acid_100g,stearic-acid_100g,arachidic-acid_100g,behenic-acid_100g,lignoceric-acid_100g,cerotic-acid_100g,montanic-acid_100g,melissic-a